**Clean and buffer MVUM road data**

Assumptions: Dataset is already filtered to a given state, and has validity filters. Can update to generate validity filters or state filters in the future.

In [2]:
import geopandas as gpd
import sys
from pathlib import Path

notebook_path = Path.cwd()
project_root = notebook_path.parent
sys.path.append(str(project_root))


from scripts import config

layer_name = 'MVUM_Roads_CO'

In [3]:
gdf = gpd.read_file(config.GDB_PATH, layer=layer_name)

print(gdf.columns.tolist())
print(gdf.loc[0])

['FID_Motor_Vehicle_Use_Map_3A_Roads__Feature_Layer_', 'OBJECTID', 'RTE_CN', 'ID', 'NAME', 'BMP', 'EMP', 'SEG_LENGTH', 'GIS_MILES', 'SYMBOL', 'MVUM_SYMBO', 'JURISDICTI', 'OPERATIONA', 'SURFACETYP', 'SYSTEM', 'SEASONAL', 'PASSENGERV', 'PASSENGE_1', 'HIGHCLEARA', 'HIGHCLEA_1', 'TRUCK', 'TRUCK_DATE', 'BUS', 'BUS_DATESO', 'MOTORHOME', 'MOTORHOME_', 'FOURWD_GT5', 'FOURWD_G_1', 'TWOWD_GT50', 'TWOWD_GT_1', 'TRACKED_OH', 'TRACKED__1', 'OTHER_OHV_', 'OTHER_OH_1', 'ATV', 'ATV_DATESO', 'MOTORCYCLE', 'MOTORCYC_1', 'OTHERWHEEL', 'OTHERWHE_1', 'TRACKED__2', 'TRACKED__3', 'OTHER_OH_2', 'OTHER_OH_3', 'ADMINORG', 'SECURITYID', 'DISTRICTNA', 'FORESTNAME', 'FIELD_ID', 'SBS_SYMBOL', 'ROUTESTATU', 'GLOBALID', 'TA_SYMBOL', 'E_BIKE_CLA', 'E_BIKE_C_1', 'E_BIKE_C_2', 'E_BIKE_C_3', 'E_BIKE_C_4', 'E_BIKE_C_5', 'SHAPELEN', 'FID_CO_State_Boundary_Buffer', 'Shape_Length', 'juris_valid', 'status_valid', 'geometry']
FID_Motor_Vehicle_Use_Map_3A_Roads__Feature_Layer_                                                   1

**Cleaning**

First, let's drop unnecessary columns.

In [4]:
gdf = gdf[['geometry', 'ID', 'NAME', 'juris_valid', 'status_valid']]
gdf.head()

,geometry,ID,NAME,juris_valid,status_valid
0,"MULTILINESTRING ((-106.93728 40.92019, -106.93...",502.1,DRIVEWAY,1,1
1,"MULTILINESTRING ((-106.56575 41.00296, -106.56...",498,HOLROYD PARK,1,1
2,"MULTILINESTRING ((-106.77865 40.36139, -106.77...",289.1,JEB'S LOOP,1,1
3,"MULTILINESTRING ((-106.13835 40.87591, -106.13...",881.1,GOVERNMENT CR,1,1
4,"MULTILINESTRING ((-106.60609 40.93417, -106.60...",660.1,HELENA,1,1


Next, we only want to keep roads with valid jurisdiction and status (National Forests and EXISTING respectively). Additional validity features can be added and included in the future.

In [5]:
gdf_valid = gdf[(gdf['juris_valid'] == '1') & (gdf['status_valid'] == '1')]

print("Occurences of valid marks = 0: ", gdf_valid[(gdf_valid['juris_valid'] == 0) | (gdf_valid['status_valid'] == 0)].shape[0])
gdf_valid.head()

Occurences of valid marks = 0:  0


,geometry,ID,NAME,juris_valid,status_valid
0,"MULTILINESTRING ((-106.93728 40.92019, -106.93...",502.1,DRIVEWAY,1,1
1,"MULTILINESTRING ((-106.56575 41.00296, -106.56...",498,HOLROYD PARK,1,1
2,"MULTILINESTRING ((-106.77865 40.36139, -106.77...",289.1,JEB'S LOOP,1,1
3,"MULTILINESTRING ((-106.13835 40.87591, -106.13...",881.1,GOVERNMENT CR,1,1
4,"MULTILINESTRING ((-106.60609 40.93417, -106.60...",660.1,HELENA,1,1


Now that we've filtered on valid rows, we can drop the validity markers.

In [6]:
gdf_cleaned = gdf_valid.drop(['juris_valid', 'status_valid'], axis=1)
gdf_cleaned.head()

,geometry,ID,NAME
0,"MULTILINESTRING ((-106.93728 40.92019, -106.93...",502.1,DRIVEWAY
1,"MULTILINESTRING ((-106.56575 41.00296, -106.56...",498,HOLROYD PARK
2,"MULTILINESTRING ((-106.77865 40.36139, -106.77...",289.1,JEB'S LOOP
3,"MULTILINESTRING ((-106.13835 40.87591, -106.13...",881.1,GOVERNMENT CR
4,"MULTILINESTRING ((-106.60609 40.93417, -106.60...",660.1,HELENA


**CRS and Buffering**

Now that the data has been cleaned, we can project it to the correct CRS and buffer it. Let's start with projection:

In [7]:
gdf_projected = gdf_cleaned.to_crs(config.BUFFER_CRS)
print(gdf_projected.crs)

EPSG:26913


Now, let's buffer using the value from our config file.

In [ ]:
gdf_buffered = gdf_projected.copy()
gdf_buffered = gdf_buffered.set_geometry(gdf_buffered.geometry.buffer(config.BUFFER_DISTANCE_ROAD_METERS))
gdf_buffered.head()

,geometry,ID,NAME
0,"POLYGON ((334065.693 4534811.837, 334064.605 4...",502.1,DRIVEWAY
1,"POLYGON ((368160.033 4540521.921, 368167.186 4...",498,HOLROYD PARK
2,"POLYGON ((348696.004 4469498.531, 348707.343 4...",289.1,JEB'S LOOP
3,"POLYGON ((403898.941 4525842.629, 403911.292 4...",881.1,GOVERNMENT CR
4,"POLYGON ((364507.431 4532586.685, 364502.751 4...",660.1,HELENA


Ready to export!

In [9]:
output_folder = config.OUTPUT_DIR / "mvum_roads_buffered"
output_folder.mkdir(parents=True, exist_ok=True)

output_fp = output_folder / "mvum_roads_buffered.gpkg"
gdf_buffered.to_file(output_fp, driver="GPKG")

print("Saved buffered MVUM roads to: ", output_fp)

Saved buffered MVUM roads to:  /Users/loganproffitt/Desktop/CampGIS.nosync/Repo/CampGIS/outputs/mvum_roads_buffered/mvum_roads_buffered.gpkg
